In [1]:
import pandas as pd
import numpy as np

# Preprocessing Sources

## Deaths dataset

In [64]:
deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')

In [65]:
deaths.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/4/21,5/5/21,5/6/21,5/7/21,5/8/21,5/9/21,5/10/21,5/11/21,5/12/21,5/13/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,2648,2654,2664,2673,2683,2686,2698,2710,2713,2721
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,2402,2403,2406,2408,2411,2412,2416,2420,2423,2426
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,3289,3299,3307,3315,3321,3328,3335,3343,3350,3355
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,127,127,127,127,127,127,127,127,127,127
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,609,618,622,628,630,633,636,639,645,649


Dropping unnecessary features as we'll work with country-wise information.

In [66]:
deaths.drop(columns=['Province/State'], inplace=True)

### Changing data display from wide format to long format

This will be useful later on for easier data viz.

It's simply a reshape of the dataframe grabbing all the dates columns and turning them into one `Date` column and using each of their values for a new column called `Deaths`.

In [67]:
dates_list = deaths.columns.drop(['Country/Region', 'Lat', 'Long' ])

deaths_df = deaths.melt(id_vars=['Country/Region', 'Lat', 'Long' ], value_vars=dates_list,
           var_name='Date', value_name='Deaths')

deaths_df.head()

,Country/Region,Lat,Long,Date,Deaths
0,Afghanistan,33.93911,67.709953,1/22/20,0
1,Albania,41.15330,20.168300,1/22/20,0
2,Algeria,28.03390,1.659600,1/22/20,0
3,Andorra,42.50630,1.521800,1/22/20,0
4,Angola,-11.20270,17.873900,1/22/20,0


As some countries have information per `Province/State` we need to group information by `Country/Region` and `Date` to get the correspondent total `Deaths`.

In [68]:
deaths_grouped = deaths_df.groupby(by=['Country/Region', 'Date'], as_index=False).agg({
    'Lat': np.mean, # We'll use the mean to get a correct location to be used in Tableau
    'Long': np.mean,
    'Deaths': sum
})

## Confirmed cases data set

Given that's the same format as in the `deaths` dataset, we'll perform the same operations.

In [70]:
confirmed = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
confirmed.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/4/21,5/5/21,5/6/21,5/7/21,5/8/21,5/9/21,5/10/21,5/11/21,5/12/21,5/13/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,60563,60797,61162,61455,61755,61842,62063,62403,62718,63045
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,131327,131419,131510,131577,131666,131723,131753,131803,131845,131890
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,122999,123272,123473,123692,123900,124104,124288,124483,124682,124889
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,13316,13340,13363,13390,13406,13423,13429,13447,13470,13470
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,27284,27529,27921,28201,28477,28740,28875,29146,29405,29695


In [ ]:
confirmed.drop(columns=['Province/State'], inplace=True)

confirmed_df = confirmed.melt(id_vars=['Country/Region', 'Lat', 'Long' ], value_vars=dates_list,
           var_name='Date', value_name='Confirmed')


confirmed_grouped = confirmed_df.groupby(by=['Country/Region', 'Date'], as_index=False).agg({
    'Lat': np.mean, # We'll use the mean to get a correct location to be used in Tableau
    'Long': np.mean,
    'Confirmed': sum
})

## Recovered cases data set

In [83]:
recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

In [85]:
recovered.drop(columns=['Province/State'], inplace=True)

recovered_df = recovered.melt(id_vars=['Country/Region', 'Lat', 'Long' ], value_vars=dates_list,
           var_name='Date', value_name='Recovered')


recovered_grouped = recovered_df.groupby(by=['Country/Region', 'Date'], as_index=False).agg({
    'Lat': np.mean, # We'll use the mean to get a correct location to be used in Tableau
    'Long': np.mean,
    'Recovered': sum
})

# Merging data

In [91]:
recovered_grouped.shape, deaths_grouped.shape, confirmed_grouped.shape 

((91776, 5), (91776, 5), (91776, 5))

In [93]:
data = recovered_grouped.merge(deaths_grouped, on=['Country/Region', 'Lat', 'Long', 'Date']).merge(confirmed_grouped, on=['Country/Region', 'Lat', 'Long', 'Date'])

In [104]:
data.sample(5)

,Country/Region,Date,Lat,Long,Recovered,Deaths,Confirmed
88268,Yemen,5/11/20,15.552727,48.516388,1,9,56
54345,Micronesia,5/21/20,7.425600,150.550800,0,0,0
53024,Mauritania,8/6/20,21.007900,-10.940800,5291,157,6444
28846,France,2/24/21,6.425300,-9.839634,260230,85473,3721061
76543,Sri Lanka,10/3/20,7.873054,80.771797,3254,13,3395


In [106]:
pd.read_html('https://www.worldometers.info/world-population/population-by-country')

HTTPError: HTTP Error 403: Forbidden